## Package

In [ ]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os.path
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.models import Sequential
from keras.models import Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

from keras.utils import plot_model
from keras import regularizers
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
#from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_uniform
import visualkeras
from keras.optimizers import Adam

from IPython.display import Markdown, display
import seaborn as sns

# from lib import proc_img
# from lib import create_gen

def printmd(string):
    #Print with Markdowns    
    display(Markdown(string))

In [ ]:
%load_ext autotime
RANDOM_STATE = 42

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
tf.test.is_gpu_available()

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train_images, train_labels, test_size=0.2, shuffle= True)

In [ ]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters

    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
# #Greedy
# learning rate=  0.001 
# dense_size_1=  64 
# dense_size_2=  64 
# s1_filter_size_1=  16 
# s2_filter_size_1=  16 
# s2_filter_size_2=  16 
# s2_filter_size_3=  128 
# s3_filter_size_1=  16 
# s3_filter_size_2=  16 
# s3_filter_size_3=  128 
# s4_filter_size_1=  16 
# s4_filter_size_2=  16 
# s4_filter_size_3=  128 
# s5_filter_size_1=  16 
# s5_filter_size_2=  16 
# s5_filter_size_3=  128 
# dp_size_1=  0.0 
# dp_size_2=  0.0 
# wd_size_1=  0.0 
# wd_size_2=  0.0

In [ ]:
# #Random Search
# s1_filter_size_1= 64
# s2_filter_size_1= 256
# s2_filter_size_2= 128
# s2_filter_size_3= 64
# s3_filter_size_1= 512
# s3_filter_size_2= 64
# s3_filter_size_3= 128
# s4_filter_size_1= 32
# s4_filter_size_2= 16
# s4_filter_size_3= 128
# s5_filter_size_1= 32
# s5_filter_size_2= 32
# s5_filter_size_3= 512
# dp_size_1= 0.4
# dense_size_1= 64
# wd_size_1= 0.0
# dense_size_2= 256
# wd_size_2= 0.0
# dp_size_2= 0.0
# learning_rate= 0.001

In [ ]:
##Bayesian
# s1_filter_size_1= 64
# s2_filter_size_1= 512
# s2_filter_size_2= 256
# s2_filter_size_3= 512
# s3_filter_size_1= 512
# s3_filter_size_2= 16
# s3_filter_size_3= 512
# s4_filter_size_1= 64
# s4_filter_size_2= 128
# s4_filter_size_3= 16
# s5_filter_size_1= 16
# s5_filter_size_2= 16
# s5_filter_size_3= 16
# dp_size_1= 0.0
# dense_size_1= 64
# wd_size_1= 0.8
# dense_size_2= 128
# wd_size_2= 0.0
# dp_size_2= 0.1
# learning_rate= 0.001

                                                                             
input_shape=(32, 32, 3)
X_input = Input(input_shape)

X = ZeroPadding2D((1, 1))(X_input)

X = Conv2D(s1_filter_size_1, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
X = BatchNormalization(axis=3, name='bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides=(2, 2))(X)

X = convolutional_block(X, f=3, filters=[s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='a', s=1)
X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='b')
X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='c')

X = convolutional_block(X, f=3, filters=[s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='a', s=2)
X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='b')
X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='c')
X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='d')

X = convolutional_block(X, f=3, filters=[s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='a', s=2)
X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='b')
X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='c')
X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='d')
X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='e')
X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='f')

X = X = convolutional_block(X, f=3, filters=[s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='a', s=2)
X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='b')
X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='c')

X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

X= Dropout(dp_size_1)(X)
X = Flatten()(X)
X=Dense(dense_size_1, activation='relu',kernel_regularizer=regularizers.l2(wd_size_1), name='fc1',kernel_initializer=glorot_uniform(seed=0))(X)
X=Dense(dense_size_2, activation='relu',kernel_regularizer=regularizers.l2(wd_size_2), name='fc2',kernel_initializer=glorot_uniform(seed=0))(X)
X= Dropout(dp_size_2)(X)
X = Dense(10,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(X)

model = Model(inputs=X_input, outputs=X, name='ResNet50')

my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]
opt = Adam(learning_rate=learning_rate_1)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                optimizer=opt,
                                metrics=['accuracy'])
model.fit(x_train,y_train,
          batch_size=64,
          epochs=100,
          validation_data=(x_valid,y_valid),
          callbacks=[my_callbacks])                                                                 